In [1]:
import json
import requests
import time
import traceback
import re
from ipywidgets import Textarea, VBox, Button, Output, IntProgress, Layout
from IPython.display import display, clear_output, HTML

# Import our custom interpreter function from the src folder
from src.interpreter import calculate_bill_from_dsl

print("✅ Setup Complete: All libraries and the custom interpreter are loaded.")

✅ Setup Complete: All libraries and the custom interpreter are loaded.


In [2]:
user_orders = [
    "We had two schnitzels at 15 euros each and a large beer for 5 euros.",
    "My order was one salad for 12 euro and a bottle of water for 4 euro.",
    "The total for our food was 50 euros, and we had a bottle of wine for 25. The service was great, so add a 10 euro tip to the bill."
]

print(f"✅ List of {len(user_orders)} user orders defined successfully.")

✅ List of 3 user orders defined successfully.


In [ ]:
def create_lark_demo():
    query_input = Textarea(
        value=user_orders[0],
        placeholder='Paste an order here...',
        description='Customer Order:',
        style={'description_width': 'initial'},
        layout={'width': '95%', 'height': '100px'}
    )

    run_button = Button(description='Generate DSL & Calculate', button_style='success', icon='play')
    progress_bar = IntProgress(value=0, min=0, max=10, description='Waiting...', bar_style='info', orientation='horizontal', layout={'visibility': 'hidden'})
    output_area = Output()

    def extract_dsl_from_string(text):
        match = re.search(r'bill\s*\{[^}]*\}', text, re.DOTALL)
        return match.group(0) if match else None

    def run_live_demo(button):
        with output_area:
            clear_output(wait=True)
            run_button.disabled = True
            progress_bar.description = 'Sending...'
            progress_bar.layout.visibility = 'visible'
            
            user_query = query_input.value
            
            # --- NEW PROMPT FOR THE UPGRADED DSL ---
            prompt = f"""
            You are a helpful assistant that translates natural language into a custom DSL.
            The DSL format is:
            bill {{
              itemName: quantity * pricePerItem
              anotherItemName: price
            }}
            
            Translate the following user order into this DSL. The item names should be simple, lowercase words.

            User Order: "{user_query}"
            
            DSL Response:
            """
            
            llm_raw_output = ""
            try:
                api_response_stream = requests.post('http://localhost:11434/api/generate', json={ "model": "llama3:8b", "prompt": prompt, "stream": True }, stream=True)
                api_response_stream.raise_for_status()
                progress_bar.description = 'LLM Generating DSL...'; progress_bar.value = 5
                
                llm_raw_output = "".join(json.loads(chunk).get('response', '') for chunk in api_response_stream.iter_lines() if chunk)
                
                progress_bar.value = 10
                
                if not llm_raw_output.strip():
                    raise ValueError("The LLM returned an empty response.")

                llm_dsl_code = extract_dsl_from_string(llm_raw_output)
                if not llm_dsl_code:
                    raise ValueError("Could not find valid DSL code in the LLM's response.")

                display(HTML("<h4>1. LLM Translation</h4>"))
                display(HTML(f"<p>The LLM translated the user's order into the following DSL code:</p><pre style='background-color:#f5f5f5; border: 1px solid #ccc; padding: 10px; border-radius: 5px;'>{llm_dsl_code}</pre>"))

                display(HTML("<h4>2. Interpreter Execution Result</h4>"))
                
                # This call now works correctly.
                dsl_result = calculate_bill_from_dsl(llm_dsl_code, 'tax_rules.dsl')

                result_html = f"""
                <p>Our custom interpreter executed the DSL code and produced the following, guaranteed-correct result:</p>
                <table style="width:100%; border-collapse: collapse;">
                  <tr style="background-color:#f2f2f2;"><th style="text-align:left; padding:8px; border: 1px solid #ddd;">Component</th><th style="text-align:right; padding:8px; border: 1px solid #ddd;">Calculated Value</th></tr>
                  <tr><td style="padding:8px; border: 1px solid #ddd;">Net Food Cost</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_result['net_food_cost']:.2f}</td></tr>
                  <tr><td style="padding:8px; border: 1px solid #ddd;">Net Drink Cost</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_result['net_drink_cost']:.2f}</td></tr>
                  <tr><td style="padding:8px; border: 1px solid #ddd;">Total Tax</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_result['total_tax']:.2f}</td></tr>
                  <tr style="font-weight:bold; background-color:#f2f2f2;"><td style="padding:8px; border: 1px solid #ddd;">FINAL BILL</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_result['final_bill']:.2f}</td></tr>
                </table>
                """
                display(HTML(result_html))

            except (ValueError, json.JSONDecodeError) as e:
                print(f"🔴 LLM HALLUCINATION DETECTED: {e}")
                print("\n--- LLM's Actual (Broken) Output ---")
                print(llm_raw_output)
            except Exception:
                print("🔴 An unexpected error occurred. Full traceback below:")
                traceback.print_exc()
            finally:
                run_button.disabled = False
                progress_bar.layout.visibility = 'hidden'
                progress_bar.value = 0

    run_button.on_click(run_live_demo)
    print("--- DSL Validator ---")
    display(VBox([query_input, run_button, progress_bar, output_area]))

# --- Run the function to display the UI ---
create_lark_demo()

--- DSL Validator ---
